In [1]:
from textblob.classifiers import NaiveBayesClassifier
from textblob import TextBlob
import pandas as pd

# https://github.com/EvertonTomalok/preprocessingtext/blob/master/README.md
from preprocessingtext import CleanSentences


pre_processing = CleanSentences()

ModuleNotFoundError: No module named 'preprocessingtext'

## DataSet de treino

In [ ]:
#  1 = Good
# -1 = Bad

data_set_treino = [
    ('Eu amo muito tudo isso', 1),
    ('Eu odeio isso', -1),
    ('Eu gostei bastante desse comentário', 1),
    ('Por favor, não me ligue mais', -1),
    ('Eu gosto de sair com meus amigos', 1),
    ('O Bolsonaro é um cara escroto', -1),
    ('Programar em python é bom demais', 1),
    ('Não existe um candidato para se votar', 1),
    ('Você não deveria fazer isso', -1),
    ('Deveriamos fazer isso mais vezes', 1),
    ('Isso não foi legal', -1),
    ('Eu amo esse saunduice', 1),
    ('Estou cansado deste trabalho', -1),
    ('Que lugar incrível', 1),
    ('Essa cerveja parece boa', 1),
    ('Não quero ir nesse restarurante', -1),
    ('Esse lugar é meu favorito', 1),
    ("Não consigo lidar com isso", -1),
    ('Que cara chato', -1),
    ('Meu chefe é horrível', -1),
    ('A cerveja esta boa', 1),
    ('Não gosto do meu trabalho', -1),
    ("Me sinto incrível hoje", 1),
    ("Não quero ir hoje", -1),
    ('Ele é meu melhor amigo', 1),
    ("Não acredito que estou fazendo isso", -1)
    
]

# Efetuando o pre-processamento do texto removendo stop words, pontuação e normalizando acentuação

In [ ]:
train_set_pos_processing = []

for element, sentimento in data_set_treino:
    new_elemento = pre_processing.stem_sentence(element, remove_stop_words=True)
    train_set_pos_processing.append((new_elemento, sentimento))


## Visualizando antes e depois do processamento

In [ ]:
data = dict()
antes = [x for x, _ in data_set_treino]
depois = list(x for x, _ in train_set_pos_processing)
sentimento = [x for _, x in data_set_treino]

data = {
    'antes': antes,
    'depois': depois,
    'sentimento': sentimento
}

df = pd.DataFrame(data)
df

## Delimitando o tamanho do data sete em 70% para treino, e 30% para teste

In [ ]:
treino = int(len(train_set_pos_processing)*0.7)


treino
# 25 * 0.7 = 17.5 => Arrendondando = 18

## Treinando o modelo com base no teorema de Naive Bayes

In [ ]:
classificador_sem_stop_words = NaiveBayesClassifier(train_set_pos_processing[:treino])

## Medindo a acurácia do modelo removendo stop words

In [ ]:
accuracy = classificador_sem_stop_words.accuracy(train_set_pos_processing[treino:])
accuracy = accuracy * 100
print('Precisão da previsão: {}%'.format(accuracy))

## Visualizando as features

In [ ]:
classificador_sem_stop_words.show_informative_features()  

<br><br><br>
# Modelo sem remoção de stop words

In [ ]:
train_set_pos_processing = []

for element, sentimento in data_set_treino:
    new_elemento = pre_processing.stem_sentence(element, remove_stop_words=False)
    train_set_pos_processing.append((new_elemento, sentimento))


## Analisando o novo data_set

In [ ]:
data = dict()
antes = [x for x, _ in data_set_treino]
depois = list(x for x, _ in train_set_pos_processing)
sentimento = [x for _, x in data_set_treino]

data = {
    'antes': antes,
    'depois': depois,
    'sentimento': sentimento
}

df = pd.DataFrame(data)
df

## Delimitando 70% e treinando o modelo

In [ ]:
treino = int(len(train_set_pos_processing)*0.7)

classificador_com_stop_words = NaiveBayesClassifier(train_set_pos_processing[:treino])

## Medindo a acurácia deste modelo

In [ ]:
## Neste modelo, mantendo as stop words, a acurácia do modelo caiu consideravelmente

accuracy = classificador_com_stop_words.accuracy(train_set_pos_processing[treino:])
accuracy = accuracy * 100
print('Precisão da previsão: {}%'.format(accuracy))

<br><br><br><br>

# Utilizando os modelos

In [ ]:
# stop_word= True (manter)
# stop_word= False (remover)

def sentimento(text, stop_word=True):
    
    bkp_text = text
    text = pre_processing.stem_sentence(text, remove_stop_words= not stop_word)
    
    
    # Definindo o classificador conforme o paramentro recebido
    if stop_word:
        classificador = classificador_com_stop_words
    else:
        classificador = classificador_sem_stop_words

    # Transformando o texto recebido em um objeto TextBlob
    blob = TextBlob(text, classifier=classificador)
    result = blob.classify()

    if result == 1:
        to_print = '\033[1;30;42m POS \033[m'
    elif result == -1:
        to_print = '\033[1;30;41m NEG \033[m'
    
    print('========================================')
    print('Frase analisada: ', bkp_text)
    print('\nEsta frase é de caráter: {}\n'.format(to_print))
    print('========================================')
    
    del(bkp_text)
    del(to_print)


<br><br>
# Utilizando o modelo retirando stop_words

In [ ]:
sentimento('Eu adorei essa noite', stop_word=False)
print()
sentimento('Eu não gostei do que você falou', stop_word=False)
print()

sentimento('Eu gostei de hoje', stop_word=False)

<br><br>
# Utilizando o modelo NÃO retirando stop_words

In [ ]:
sentimento('Eu adorei essa noite', stop_word=True)
print()
sentimento('Eu não gostei do que você falou', stop_word=True)
# previsão falso positiva